# 1. Package 설치

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone

# 2. Knowledge Base 구성을 위한 데이터 생성

- 4. LangChain을 활용한 Vector Database 변경 (Chroma -> Pinecone)과 동일함

In [13]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 소득세법
# document_name = './tax_with_markdown.docx'
# 민법
# document_name = './civil_with_markdown.docx'
# 형법
# document_name = './criminal_with_markdown.docx'
# 도로교통법
# document_name = './traffic_with_markdown.docx'
# 근로기준법
document_name = './labor_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [14]:
len_list = len(document_list)
len_list

29

In [15]:
document_list[len_list - 10]

Document(metadata={'source': './labor_with_markdown.docx'}, page_content='제70조(야간근로와 휴일근로의 제한) ① 사용자는 18세 이상의 여성을 오후 10시부터 오전 6시까지의 시간 및 휴일에 근로시키려면 그 근로자의 동의를 받아야 한다.\n\n② 사용자는 임산부와 18세 미만자를 오후 10시부터 오전 6시까지의 시간 및 휴일에 근로시키지 못한다. 다만, 다음 각 호의 어느 하나에 해당하는 경우로서 고용노동부장관의 인가를 받으면 그러하지 아니하다.<개정 2010. 6. 4.>\n\n1. 18세 미만자의 동의가 있는 경우\n\n2. 산후 1년이 지나지 아니한 여성의 동의가 있는 경우\n\n3. 임신 중의 여성이 명시적으로 청구하는 경우\n\n③ 사용자는 제2항의 경우 고용노동부장관의 인가를 받기 전에 근로자의 건강 및 모성 보호를 위하여 그 시행 여부와 방법 등에 관하여 그 사업 또는 사업장의 근로자대표와 성실하게 협의하여야 한다.<개정 2010. 6. 4.>\n\n\n\n제71조(시간외근로) 사용자는 산후 1년이 지나지 아니한 여성에 대하여는 단체협약이 있는 경우라도 1일에 2시간, 1주에 6시간, 1년에 150시간을 초과하는 시간외근로를 시키지 못한다. <개정 2018. 3. 20.>\n\n\n\n제72조(갱내근로의 금지) 사용자는 여성과 18세 미만인 사람을 갱내(坑內)에서 근로시키지 못한다. 다만, 보건ㆍ의료, 보도ㆍ취재 등 대통령령으로 정하는 업무를 수행하기 위하여 일시적으로 필요한 경우에는 그러하지 아니하다. <개정 2020. 5. 26.>\n\n\n\n제73조(생리휴가) 사용자는 여성 근로자가 청구하면 월 1일의 생리휴가를 주어야 한다.\n\n\n\n제74조(임산부의 보호) ① 사용자는 임신 중의 여성에게 출산 전과 출산 후를 통하여 90일(한 번에 둘 이상 자녀를 임신한 경우에는 120일)의 출산전후휴가를 주어야 한다. 이 경우 휴가 기간의 배정은 출산 후에 45일(한 번에 둘 이상 자녀를

In [16]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
%pip install --upgrade jupyter

In [ ]:
%pip install --upgrade ipywidgets

In [17]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

is_create = False
# 소득세법
# index_name = 'tax-markdown-index'
# 민법
# index_name = 'civil-markdown-index'
# 형법
# index_name = 'criminal-markdown-index'
# 도로교통법
# index_name = 'traffic-markdown-index'
# 도로교통법
index_name = 'labor-markdown-index'

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 데이터를 추가할 때는 `from_documents()` 데이터를 추가한 이후에는 `from_existing_index()`를 사용합니다
if is_create:
    database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
else:
    database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

In [18]:
# 소득세법
# query = '연봉 5천만원인 직장인의 소득세는?'
# 민법
# query = '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?'
# 형법
# query = '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형법적 처벌을 받을 수 있나요?'
# 도로교통법
# query = '음주운전 단속 기준은 어떻게 되나요?'
# 근로기준법
query = '임금 체불 시 어떻게 대응해야 하나요?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [19]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './labor_with_markdown.docx'}, page_content='2. 「민사집행법」 제56조제3호에 따른 확정된 지급명령, 하수급인의 근로자에게 하수급인에 대하여 임금채권이 있음을 증명하는 같은 법 제56조제4호에 따른 집행증서,「소액사건심판법」 제5조의7에 따라 확정된 이행권고결정, 그 밖에 이에 준하는 집행권원이 있는 경우\n\n3. 하수급인이 그가 사용한 근로자에 대하여 지급하여야 할 임금채무가 있음을 직상 수급인에게 알려주고, 직상 수급인이 파산 등의 사유로 하수급인이 임금을 지급할 수 없는 명백한 사유가 있다고 인정하는 경우\n\n② 「건설산업기본법」 제2조제10호에 따른 발주자의 수급인(이하 “원수급인”이라 한다)으로부터 공사도급이 2차례 이상 이루어진 경우로서 하수급인(도급받은 하수급인으로부터 재하도급 받은 하수급인을 포함한다. 이하 이 항에서 같다)이 사용한 근로자에게 그 하수급인에 대한 제1항제2호에 따른 집행권원이 있는 경우에는 근로자는 하수급인이 지급하여야 하는 임금(해당 건설공사에서 발생한 임금으로 한정한다)에 해당하는 금액을 원수급인에게 직접 지급할 것을 요구할 수 있다. 원수급인은 근로자가 자신에 대하여 「민법」 제404조에 따른 채권자대위권을 행사할 수 있는 금액의 범위에서 이에 따라야 한다.<개정 2011. 5. 24.>\n\n③ 직상 수급인 또는 원수급인이 제1항 및 제2항에 따라 하수급인이 사용한 근로자에게 임금에 해당하는 금액을 지급한 경우에는 하수급인에 대한 하도급 대금 채무는 그 범위에서 소멸한 것으로 본다.\n\n[본조신설 2007. 7. 27.]\n\n\n\n제45조(비상시 지급) 사용자는 근로자가 출산, 질병, 재해, 그 밖에 대통령령으로 정하는 비상(非常)한 경우의 비용에 충당하기 위하여 임금 지급을 청구하면 지급기일 전이라도 이미 제공한 근로에 대한 임금을 지급하여야 한다.\n\n\n\n제46조(휴업수당) ① 사용자의 귀책사유로 휴업하는 경우에 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [20]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [22]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [23]:
ai_message = qa_chain.invoke({"query": query})

In [24]:
ai_message

{'query': '임금 체불 시 어떻게 대응해야 하나요?',
 'result': '임금 체불 시, 직상 수급인에게 하수급인의 임금채무를 알려야 하며, 직상 수급인이 파산 등의 사유로 하수급인이 임금을 지급할 수 없는 명백한 사유가 있다고 인정하는 경우에는 원수급인에게 직접 지급을 요구할 수 있습니다. 또한, 고용노동부장관에게 체불사업주의 명단 공개를 요청할 수 있습니다. 마지막으로, 근로자의 임금은 사용자의 총재산에서 다른 채권에 우선하여 변제됩니다.'}